# Randomness and reproducibility

Random numbers and [stochastic processes](http://www2.econ.iastate.edu/tesfatsi/ace.htm#Stochasticity)
are essential to most agent-based models.
[Pseudo-random number generators](https://en.wikipedia.org/wiki/Pseudorandom_number_generator)
can be used to create numbers in a sequence that appears 
random but is actually a deterministic sequence based on an initial seed value.
In other words, the generator will produce the same pseudo-random sequence 
over multiple runs if it is given the same seed at the beginning.
Note that is possible that the generators will draw the same number repeatedly, 
as illustrated in this [comic strip](https://dilbert.com/strip/2001-10-25) from Scott Adams:

![Alt text](graphics/dilbert_rng.gif)

In [1]:
import agentpy as ap
import numpy as np
import random

## Random number generators

Agentpy models contain two internal pseudo-random number generators with different features:

- `Model.random` is an instance of `random.Random` (more info [here](https://realpython.com/python-random/))
- `Model.nprandom` is an instance of `numpy.random.Generator` (more info [here](https://numpy.org/devdocs/reference/random/index.html))

To illustrate, let us define a model that uses both generators to draw a random integer:

In [2]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.x = self.random.randint(0, 99)
        self.y = self.nprandom.integers(99)
        self.report(['x', 'y'])

If we run this model multiple times, we will likely get a different series of numbers:

In [3]:
exp = ap.Experiment(RandomModel, iterations=5)
results = exp.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.042848


In [4]:
results.reporters

x   y
sample_id iteration        
0         0          40   2
          1          32  79
          2          77  10
          3          35  37
          4           9  81

## Defining custom seeds

If we want the results to be reproducible, 
we can define a parameter `seed` that 
will be used automatically at the beginning of `Model.run`
to initialize both generators.

In [5]:
parameters = {'seed': 42}
exp = ap.Experiment(RandomModel, parameters, iterations=5)
results = exp.run()

Scheduled runs: 5
Completed: 5, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.040061


Now, we get the same set of numbers during every iteration:

In [6]:
results.reporters

x   y
sample_id iteration        
0         0          35  39
          1          35  39
          2          35  39
          3          35  39
          4          35  39

## Stochastic processes for agent groups

Let us now look at some stochastic operations that are often used in agent-based models. 
To start, we create a list of five agents:

In [19]:
model = ap.Model()
agents = ap.AgentList(model, 5)

In [20]:
agents

AgentList (5 objects)

If we look at the agent's ids, we see that they have been created in order:

In [21]:
agents.id

[1, 2, 3, 4, 5]

To shuffle this list, we can use `AgentList.shuffle`:

In [22]:
agents.shuffle().id

[3, 5, 1, 2, 4]

To create a random subset, we can use `AgentList.random`:

In [26]:
agents.random(3).id

[1, 5, 4]

And if we want it to be possible to select the same agent more than once:

In [27]:
agents.random(6, replace=True).id

[2, 5, 1, 1, 4, 2]

## Varying seeds

### Variations in parameter samples

To define a custom set of seeds, we can sample the seed like any other parameter:

In [110]:
parameters1 = {'p': ap.Values(0, 1), 'seed': ap.Values(0, 1)}
sample1 = ap.Sample(parameters1)

In [111]:
list(sample1)

[{'p': 0, 'seed': 0},
 {'p': 0, 'seed': 1},
 {'p': 1, 'seed': 0},
 {'p': 1, 'seed': 1}]

Alternatively, we we can pass an argument `'seed'` to `Sample`
to generate random seeds for each parameter combination in the sample.

In [142]:
parameters2 = {'p': ap.Values(0, 1)}
sample2 = ap.Sample(parameters2, seed=1)

In [143]:
list(sample2)

[{'p': 0, 'seed': 272996653310673477252411125948039410165},
 {'p': 1, 'seed': 40125655066622386354123033417875897284}]

This will always produce the same set of random seeds:

In [144]:
sample3 = ap.Sample(parameters2, seed=1)

In [145]:
list(sample3)

[{'p': 0, 'seed': 272996653310673477252411125948039410165},
 {'p': 1, 'seed': 40125655066622386354123033417875897284}]

### Variations in experiment iterations

Returning to the first sample, let us run an experiment.
Every iteration with the same seed parameter will have the same results.

In [116]:
exp = ap.Experiment(RandomModel, sample1, iterations=2)
results = exp.run()

Scheduled runs: 8
Completed: 8, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.051122


In [117]:
results.arrange_reporters()

,sample_id,iteration,x,y,p,seed
0,0,0,53,19,0,0
1,0,1,53,19,0,0
2,1,0,97,38,0,1
3,1,1,97,38,0,1
4,2,0,53,19,1,0
5,2,1,53,19,1,0
6,3,0,97,38,1,1
7,3,1,97,38,1,1


Alternatively, we can initialize the experiment with `random=True`,
which will use the seed parameter to generate new seeds 
for each iteration with that parameter combination.

In [118]:
exp = ap.Experiment(RandomModel, sample1, iterations=2, random=True)
results = exp.run()

Scheduled runs: 8
Completed: 8, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.034919


Parameter combinations with the same iteration & seed will now have the same random output:

In [119]:
results.arrange_reporters()

,sample_id,iteration,x,y,p,seed
0,0,0,68,31,0,0
1,0,1,55,30,0,0
2,1,0,4,25,0,1
3,1,1,10,55,0,1
4,2,0,68,31,1,0
5,2,1,55,30,1,0
6,3,0,4,25,1,1
7,3,1,10,55,1,1


## Agent-specific generators

For more advanced applications, we can create seperate generators for each object.
We can ensure that the seeds of each object follow a controlled pseudo-random sequence by using the models' main generator to generate the seeds.

In [135]:
class RandomAgent(ap.Agent):
    
    def setup(self):
        seed = self.model.random.getrandbits(128) # Seed from model
        self.random = random.Random(seed)  # Create agent generator
        self.x = self.random.random()  # Create a random number
        
class MultiRandomModel(ap.Model):
    
    def setup(self):
        self.agents = ap.AgentList(self, 2, RandomAgent)
        self.agents.record('x')

In [133]:
parameters = {'seed': 42}
exp = ap.Experiment(MultiRandomModel, parameters, iterations=2, record=True)
results = exp.run()

Scheduled runs: 2
Completed: 2, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.028399


In [134]:
results.variables.RandomAgent

x
sample_id iteration obj_id t          
0         0         1      0  0.414688
                    2      0  0.591608
          1         1      0  0.414688
                    2      0  0.591608

Alternatively, we can also have each agent start from the same seed:

In [136]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        self.random = random.Random(self.p.agent_seed)  # Create agent generator
        self.x = self.random.random()  # Create a random number
        
class MultiRandomModel2(ap.Model):
    
    def setup(self):
        self.agents = ap.AgentList(self, 2, RandomAgent2)
        self.agents.record('x')

In [139]:
parameters = {'agent_seed': 42}
exp = ap.Experiment(MultiRandomModel2, parameters, iterations=2, record=True)
results = exp.run()

Scheduled runs: 2
Completed: 2, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.019749


In [141]:
results.variables.RandomAgent2

x
sample_id iteration obj_id t          
0         0         1      0  0.639427
                    2      0  0.639427
          1         1      0  0.639427
                    2      0  0.639427